In [19]:
from collections import Counter
import bs4 as bs
import pickle
import requests
import datetime as dt
import numpy as np
import os
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader as pdr
from datetime import datetime
import yfinance as yf
from pandas_datareader import data as pdr

import matplotlib.pyplot as plt
from matplotlib import style

import seaborn as sns

style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier


In [2]:
import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('\n','')
        if "." in ticker:
            ticker = ticker.replace('.','-')
            print('ticker replaced to', ticker) 
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

#save_sp500_tickers()

def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            #df = df.drop("Symbol", axis=1)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


#get_data_from_yahoo()


In [3]:
def compile_data():
    with open("sp500tickers.pickle","rb") as f:
        tickers=pickle.load(f)
    
    main_df=pd.DataFrame()
    
    for count,ticker in enumerate(tickers):
        df=pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date',inplace=True)
        
        df.rename(columns={'Adj Close':ticker},inplace=True)
        df.drop(['Open','High','Low','Close','Volume'],1,inplace=True)
        
        if main_df.empty:
            main_df=df
        else:
            main_df=main_df.join(df,how='outer')
        
        if count%10==0:
            print(count)
            
    main_df.to_csv("sp500_joined_closes.csv")  
#compile_data()


In [4]:
def visualize_data():
    df=pd.read_csv('sp500_joined_closes.csv')
    #df['AAPL'].plot
    #plt.show()
    df_corr=df.corr()
    print(df_corr.head())
    
    data = df_corr.values
    fig=plt.figure()
    
    ax= fig.add_subplot(1,1,1)
    
    heatmap=ax.pcolor(data,cmap=plt.cm.RdYlGn)
    fig.colorbar(heatmap)
    ax.set_xticks(np.arange(data.shape[0]+0.5,minor=False))
    ax.set_yticks(np.arange(data.shape[1]+0.5,minor=False))
    ax.invert_yaxis()
    ax.axis.tick_top()
    
    column_labels=df_corr.columns
    row_labels=df_corr.index
    
    ax.set_xticklabels(column_labels)
    ax.set_yticklabels(row_labels)
    
    plt.xticks(rotation=90)
    heatmap.set_clim(-1,1)
    plt.tight_layout()
    plt.show()
    
#visualize_data()

In [24]:
hm_days=7
def process_data_for_labels(ticker):
    hm_days=7
    df=pd.read_csv('sp500_joined_closes.csv',index_col=0)
    tickers=df.columns.values.tolist()
    
    df.fillna(0,inplace=True)
    
    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)]= (df[ticker].shift(-i)-df[ticker] )/df[ticker]*100
    
    df.fillna(0,inplace=True)
    return tickers,df

def buy_sell_hold(*args):
    requirement=.5
    cols=[c for c in args]
    for col in cols:
        if col>requirement:
            return 1
        if col<-requirement:
            return -1
        return 0

def extract_featuresets(ticker):
    tickers,df=process_data_for_labels(ticker)
    
    df['{}_target'.format(ticker)]=list(map(buy_sell_hold,
                                           *[df['{}_{}d'.format(ticker,i)]for i in range(1,hm_days+1)]))
    vals=df['{}_target'.format(ticker)].values.tolist()
    str_vals=[str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    
    df.fillna(0,inplace=True)
    
    df=df.replace([np.inf,-np.inf],np.nan)
    df.dropna(inplace=True)
    
    df_vals=df[[ticker for ticker in tickers]].pct_change()
    df_vals=df_vals.replace([np.inf,-np.inf],0)
    df_vals.fillna(0,inplace=True)
    
    X=df_vals.values
    y=df['{}_target'.format(ticker)].values
    
    return X,y,df
#extract_featuresets('MMM')

def do_ml(ticker):
    X,y,df=extract_featuresets(ticker)
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
   
    clf=VotingClassifier([('lsvc',svm.LinearSVC()),
                         ('knn',neighbors.KNeighborsClassifier()),
                        ('rfor',RandomForestClassifier())])
    
    clf.fit(X_train,y_train)
    
    
    confidence=clf.score(X_test,y_test)
    
    print('Accuracy',confidence)
    
    predictions=clf.predict(X_test)
    
    print("Predicted spread:",Counter(predictions))
    
    return confidence
 
    
do_ml('BAC')  



Data spread: Counter({'1': 1008, '-1': 953, '0': 737})


C:\Users\sarad\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy 0.35555555555555557
Predicted spread: Counter({-1: 323, 1: 236, 0: 116})


0.35555555555555557